# Tarea 5

**Envío del PR inicial:** 15 de octubre

**Aceptación del PR:** 21 de octubre

NOTA: Esta tarea debe entregarse en equipo.

## 1

Llamemos $c_n$ el valor del parámetro $c$ donde ocurre la bifurcación de doblamiento de periodo para el mapeo $Q_c(x)=x^2+c$, donde la órbita de periodo $2^n$ nace. Como hemos visto en notebooks anteriores, tenemos que $c_0=1/4$ marca la aparición del atractor de periodo $2^0=1$, $c_1=-1/4$ corresponde a la aparición del atractor de periodo $2^1=2$, $c_2=-3/4$ a la aparición del atractor de periodo $2^2=4$, etc. 

A partir de estos valores y otros que calcularán (al menos deben encontrar $c_6$), definimos la secuencia: $\{f_0, f_1, f_2, \dots\}$, donde

\begin{equation}
f_n = \frac{c_n-c_{n+1}}{c_{n+1}-c_{n+2}} .
\end{equation}

La pregunta es, ¿a qué valor converge esta secuencia?, es decir, dar una estimación de $f_\infty$.


*Hint:* Para realizar este ejercicio deben calcular el atractor para varios valores de $c$, de tal manera que puedan aislar las órbitas de periodo $2^p$ y de ahí determinar varios valores $c_n$. Se requerir suficiente cuidado para obtener una buena aproximación de $c_n$. 
Una opción se basa en recordar/usar que las bifurcaciones de doblamiento de periodo ocurren cuando los puntos de la órbita de periodo $p$ se tornan en repulsores, es decir, $(Q_c^p)'(x)=-1$. Esta opción, entonces, involucra obtener los valores $c_n$ a partir de órbitas periódicas de periodo $2^n$ usando los polinomios $Q_c^{2^p}(x)$ y diferenciación automática.

In [24]:
"""
    ciclosestables!(xx, f, cc, nit, nout)

Esta función itera el mapeo `f`, de una variable, `nit+nout` veces, 
usando como condición inicial `x0=0`; los últimos `nout` iterados 
actualizan al vector `xx` que tiene longitud `nout`. `cc` es el valor
del parámetro del mapeo `f`. El mapeo `f` debe ser definido de 
tal manera que `f(x0, cc)` tenga sentido. La idea es que los últimos 
`nout` iterados reflejen los ciclos estables del mapeo `f`.
"""
function ciclosestables!(xx, f, cc, nit, nout)
    @assert (nit > 0) && (nout > 0)
    
    # Primeros nit iterados
    x0 = 0.0
    for it = 1:nit
        x0 = f(x0, cc)
    end
    
    # Se guardan los siguientes nout iterados
    for it = 1:nout
        x0 = f(x0, cc)
        @inbounds xx[it] = x0
    end
    
    nothing
end

ciclosestables!

In [5]:
include("DualNum.jl")

Main.DualNum

In [16]:
using Plots

┌ Info: Recompiling stale cache file C:\Users\Carlos Borja\.julia\compiled\v1.0\Plots\ld3vC.ji for Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1184
┌ Warning: Module Compat with build ID 1372881305922 is missing from the cache.
│ This may mean Compat [34da2185-b29b-5c13-b0c7-acf172513d20] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:941
┌ Warning: Module Compat with build ID 1372881305922 is missing from the cache.
│ This may mean Compat [34da2185-b29b-5c13-b0c7-acf172513d20] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:941
┌ Info: Precompiling Showoff [992d4aef-0814-514b-bc4d-f2e9a6c4116f]
└ @ Base loading.jl:1186
┌ Warning: Module Compat with build ID 1372881305922 is missing from the cache.
│ This may mean Compat [34da2185-b29b-5c13-b0c7-acf172513d20] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:941
┌ Info: Recompiling stale c

In [40]:
"""
    diag_bifurc(f, nit, nout, crange)

Itera el mapeo `f` `nit+nout` veces y regresa una matriz
cuya columna `i` tiene los últimos `nout` iterados del mapeo
para el valor del parámetro del mapeo `crange[i]`.

La función `f` debe ser definida de tal manera que `f(x0, c)` 
tenga sentido.
"""
function diag_bifurc(f, nit, nout, crange)
#     xx = Vector{Float64}(nout)
    ff = Array{Float64,2}(undef, nout, length(crange))
    
    for ic in eachindex(crange)
        c = crange[ic]
       ciclosestables!(view(ff, :, ic), f, c, nit, nout)
#         ff[:,ic] = xx
    end
    
    return ff
end

diag_bifurc

In [38]:
Qc(x,c) = x^2 + c

Qc (generic function with 1 method)

In [53]:
a=-0.74:-1/2^10:-0.76
f = diag_bifurc(Qc,100,4,a)

4×21 Array{Float64,2}:
 -0.529891  -0.532881  -0.535984  …  -0.595679  -0.599589  -0.603469
 -0.459216  -0.457014  -0.454674     -0.402745  -0.399048  -0.395356
 -0.529121  -0.532115  -0.535225     -0.595375  -0.599316  -0.603225
 -0.460031  -0.45783   -0.455487     -0.403107  -0.399375  -0.395651

In [79]:
for c in a
    df = Qc(DualNum.dual(f[1]),c)
    print(df)
end

-0.45921571176780396 -1.0597816534214886 ϵ
-0.46019227426780396 -1.0597816534214886 ϵ
-0.46116883676780396 -1.0597816534214886 ϵ
-0.46214539926780396 -1.0597816534214886 ϵ
-0.46312196176780396 -1.0597816534214886 ϵ
-0.46409852426780396 -1.0597816534214886 ϵ
-0.46507508676780396 -1.0597816534214886 ϵ
-0.46605164926780396 -1.0597816534214886 ϵ
-0.46702821176780396 -1.0597816534214886 ϵ
-0.46800477426780396 -1.0597816534214886 ϵ
-0.46898133676780396 -1.0597816534214886 ϵ
-0.46995789926780396 -1.0597816534214886 ϵ
-0.47093446176780396 -1.0597816534214886 ϵ
-0.47191102426780396 -1.0597816534214886 ϵ
-0.47288758676780396 -1.0597816534214886 ϵ
-0.47386414926780396 -1.0597816534214886 ϵ
-0.47484071176780396 -1.0597816534214886 ϵ
-0.47581727426780396 -1.0597816534214886 ϵ
-0.47679383676780396 -1.0597816534214886 ϵ
-0.47777039926780396 -1.0597816534214886 ϵ
-0.47874696176780396 -1.0597816534214886 ϵ


In [74]:
g(x) = x^2-10

g (generic function with 1 method)

In [78]:
Qc(DualNum.dual(1),-10)

-9.0 + 2.0 ϵ


## 2

Repitan el ejercicio anterior para el mapeo $S_c(x) = c \sin(x)$. 

- ¿Cómo se comparan los valores obtenidos de $f_n$? 

- ¿Qué interpretación le pueden dar a este resultado, en comparación del ejercicio anterior?

## 3

Como se ve en la Fig. 1 del diagrama de bifurcaciones de $Q_c$, $x=0$ pertenece a un ciclo de periodo $2^n$ para ciertos valores $C_n$ del parámetro. Dichos valores son *especiales*, ya que el hecho de que $x=0$ pertenezca a un ciclo de periodo $2^n$ define los llamados *ciclos superestable*, donde tenemos $(Q^{2^p}_{C_n})'(0)=0$.

- ¿A qué converge la secuencia $f_n$, definida ahora a partir de los valores $C_n$.

- De los $2^p$ puntos del ciclo de periodo $2^p$, es decir, $\{0, p_1, \dots p_{2^{n-1}}\,\}$ hay uno (distinto del 0) cuya distancia a 0 es la menor; a esa distancia la identificaremos como $d_n$. Calcular numéricamente a qué converge la secuencia $d_n/d_{n+1}$.